<a href="https://colab.research.google.com/github/ahmedmabrouk24/Nueral-Network/blob/main/Neural_Network_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tflearn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 747.2 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=4606bcd606c5b6adbb873b6fb41d4604827a86b3eb568c093583645600f2d70a
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install imgaug

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [4]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers,models,Sequential
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense, Conv2D, Activation, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.saving.save import load_model
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
import tensorflow as tf
import cv2
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
from PIL import Image
import shutil
import json
from keras.applications.vgg16 import VGG16

In [5]:

TRAIN_DIR = '/kaggle/input/nn23-sports-image-classification/Train'
TEST_DIR = '/kaggle/input/nn23-sports-image-classification/Test'
train_images=[]
test_images=[]
val_images=[]
SPORTS_DIR = []
SPORTS_DIR.append('/kaggle/working/football')
SPORTS_DIR.append('/kaggle/working/swimming')
SPORTS_DIR.append('/kaggle/working/rowing')
SPORTS_DIR.append('/kaggle/working/yoga')
SPORTS_DIR.append('/kaggle/working/basketball')
SPORTS_DIR.append('/kaggle/working/tennis')
IMG_SIZE = 224
num_classes = 6
MODEL_NAME = 'sports_classification-cnn'

In [6]:
def create_classes_files():
    if (os.path.exists('/kaggle/working/football') == False):os.mkdir('/kaggle/working/football')
    if (os.path.exists('/kaggle/working/swimming') == False):os.mkdir('/kaggle/working/swimming')
    if (os.path.exists('/kaggle/working/rowing') == False):os.mkdir('/kaggle/working/rowing')
    if (os.path.exists('/kaggle/working/yoga') == False):os.mkdir('/kaggle/working/yoga')
    if (os.path.exists('/kaggle/working/basketball') == False):os.mkdir('/kaggle/working/basketball')
    if (os.path.exists('/kaggle/working/tennis') == False):os.mkdir('/kaggle/working/tennis')
    for img in tqdm(os.listdir(TRAIN_DIR)):
        g = img.split('_')
        img_name = g[0]
        path = os.path.join(TRAIN_DIR, img)
        img_data = Image.open(path) 
        img_data = img_data.resize((IMG_SIZE, IMG_SIZE))
        if (img_name == 'Basketball'):
            img_data = img_data.save('/kaggle/working/basketball/' + img)
        elif(img_name == 'Football'):
            img_data = img_data.save('/kaggle/working/football/' + img)
        elif(img_name == 'Rowing'):
            img_data = img_data.save('/kaggle/working/rowing/' + img)
        elif(img_name == 'Yoga'):
            img_data = img_data.save('/kaggle/working/yoga/' + img)
        elif(img_name == 'Swimming'):
            img_data = img_data.save('/kaggle/working/swimming/' + img)
        elif(img_name == 'Tennis'):
            img_data = img_data.save('/kaggle/working/tennis/' + img)
create_classes_files()

100%|██████████| 1681/1681 [00:27<00:00, 61.35it/s]


In [7]:
def create_validation_folder():
    if (os.path.exists('/kaggle/working/validation_data') == False):os.mkdir('/kaggle/working/validation_data')
    for f in SPORTS_DIR:
        cnt = 0
        for img in tqdm(os.listdir(f)):
            cnt+=1
            if (cnt == 31):break
            path = os.path.join(f, img)
            img_data = Image.open(path) 
            image = img.split('.')
            img_data = img_data.resize((IMG_SIZE, IMG_SIZE))
            img_data = img_data.save('/kaggle/working/validation_data/'+ image[0] + '.jpg')
            os.remove(path)
create_validation_folder()

 16%|█▌        | 30/185 [00:00<00:00, 369.11it/s]


In [8]:
def convert_all_image_to_jpg(path_file):
    for img in tqdm(os.listdir(path_file)):
        x = img.split(".")
        path = path_file + '/' + img
        name = path_file + '/' + x[0]
        im = Image.open(path)
        im.convert('RGB').save(name + ".jpg")
        if (x[1] != 'jpg'):
             os.remove(path) 
                
for f in SPORTS_DIR:
    convert_all_image_to_jpg(f)
convert_all_image_to_jpg('/kaggle/working/validation_data')

100%|██████████| 180/180 [00:00<00:00, 389.81it/s]


In [9]:
def augmentation(input_img):
        list_data=[]

    #Crop
        crop2 = iaa.Crop(percent=(0, 0.2)) 
        input_crop2 = crop2.augment_image(input_img)
        list_data.append(input_crop2)
    #rotation
        rot3 = iaa.Affine(rotate=(-40,20))
        input_rot3 = rot3.augment_image(input_img)
        list_data.append(input_rot3)
    #blur
        blur=iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5)))
        input_blur=blur.augment_image(input_img)
        list_data.append(input_blur)

    #noise
        noise2=iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
        input_noise2=noise2.augment_image(input_img)
        list_data.append(input_noise2)
        aug=iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
        )
        input_aug=aug.augment_image(input_img)
        list_data.append(input_aug)
    
        return list_data

In [10]:
def balance_data():
    for f in SPORTS_DIR:
        cnt = 0
        dir_of_sport = []
        for img in tqdm(os.listdir(f)):
            path = os.path.join(f, img)
            cnt+=1
            dir_of_sport.append(path)
        for img in dir_of_sport:
            path = img
            img_data = cv2.imread(path,1)
            img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)
            img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
            list_aug_data = augmentation(img_data)
            for lst in list_aug_data:
                cnt+=1
                if (cnt > 500):break
                aug_img = Image.fromarray(lst)
                if (f == '/kaggle/working/football'):
                    aug_img = aug_img.save('/kaggle/working/football/'+ 'Football' + str(cnt) + '.jpg')
                    
                elif(f == '/kaggle/working/swimming'):
                    aug_img = aug_img.save('/kaggle/working/swimming/'+ 'Swimming' + str(cnt) + '.jpg')
                    
                elif(f == '/kaggle/working/rowing'):
                    aug_img = aug_img.save('/kaggle/working/rowing/'+ 'Rowing' + str(cnt) + '.jpg')
                    
                elif(f == '/kaggle/working/yoga'):
                    aug_img = aug_img.save('/kaggle/working/yoga/'+ 'Yoga' + str(cnt) + '.jpg')
                    
                elif(f == '/kaggle/working/basketball'):
                    aug_img = aug_img.save('/kaggle/working/basketball/'+ 'Basketball' + str(cnt) + '.jpg')
                    
                elif(f == '/kaggle/working/tennis'):
                    aug_img = aug_img.save('/kaggle/working/tennis/'+ 'Tennis' + str(cnt) + '.jpg')
                    
            if (cnt > 500):break
            
balance_data()           

100%|██████████| 155/155 [00:00<00:00, 284515.15it/s]


In [11]:
def create_label(image_name):
    img = image_name[0:4]
    if img == 'Bask':
        return 0
    elif img == 'Foot':
        return 1
    elif img == 'Rowi':
        return 2
    elif img == 'Swim':
        return 3
    elif img == 'Tenn':
        return 4
    elif img == 'Yoga':
        return 5

In [12]:

def create_train_folder():
    if (os.path.exists('/kaggle/working/train_data') == False):os.mkdir('/kaggle/working/train_data')
    for f in SPORTS_DIR:
        for img in tqdm(os.listdir(f)):
            path = os.path.join(f, img)
            img_data = Image.open(path)
            img_data = img_data.save('/kaggle/working/train_data/' + img)
create_train_folder()

100%|██████████| 500/500 [00:01<00:00, 381.58it/s]


In [13]:
NEW_TRAIN_DIR = "/kaggle/working/train_data"
NEW_VALIDATION_DIR = "/kaggle/working/validation_data"

In [14]:
def create_train_data():
    training_data = []
    t = 0
    for img in tqdm(os.listdir(NEW_TRAIN_DIR)):
        path = os.path.join(NEW_TRAIN_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        training_data.append([np.array(img_data), label_list])

        #append augmentation to train data
#         aug_list = augmentation(img_data);
#         for aug in aug_list:
#             training_data.append([np.array(aug), label_list])

    print("Train size = " , len(training_data))
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [15]:
column1=[]
def create_test_data():
    test_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_data = cv2.imread(path,1)
        column1.append(img)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        test_data.append([np.array(img_data), 0])

    np.save('test_data.npy', test_data)
    return test_data

In [16]:
def create_validation_data():
    validation_data = []
    for img in tqdm(os.listdir(NEW_VALIDATION_DIR)):
        path = os.path.join(NEW_VALIDATION_DIR, img)
        img_data = cv2.imread(path,1)
        img_data = cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB)

        label_list = create_label(img)
        img_data = cv2.resize(img_data, (IMG_SIZE, IMG_SIZE))
        validation_data.append([np.array(img_data), label_list])

    shuffle(validation_data)
    np.save('validation_data.npy', validation_data)
    return validation_data

In [17]:
if (os.path.exists('train_data.npy')):
    all_images = np.load('train_data.npy',allow_pickle=True)
else: 
    all_images = create_train_data()
    
train_images.clear();
print("all images = ",len(all_images))
for i in range(0 , len(all_images)):
    train_images.append(all_images[i]);

X_train = np.array([i[0] for i in train_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_images]).reshape(-1,)

#==============================================================================================

if (os.path.exists('validation_data.npy')):
    all_images_val = np.load('validation_data.npy',allow_pickle=True)
else: 
    all_images_val = create_validation_data()
val_images.clear();
print("all images = ",len(all_images_val))
for i in range(0 , len(all_images_val)):
    val_images.append(all_images_val[i]);

X_val = np.array([i[0] for i in val_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_val = np.array([i[1] for i in val_images]).reshape(-1,)


#==============================================================================================

if (os.path.exists('test_data.npy')):
    all_images_test = np.load('test_data.npy',allow_pickle=True)
else: 
    all_images_test = create_test_data()
test_images.clear()
print("all images = ",len(all_images_test))
for i in range(0 , len(all_images_test)):
    test_images.append(all_images_test[i]);

X_test = np.array([i[0] for i in test_images]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y_test = np.array([i[1] for i in test_images]).reshape(-1,)


100%|██████████| 3000/3000 [00:02<00:00, 1295.50it/s]
/opt/conda/lib/python3.7/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Train size =  3000
all images =  3000


100%|██████████| 180/180 [00:00<00:00, 1246.50it/s]


all images =  180


100%|██████████| 688/688 [00:09<00:00, 75.11it/s] 


all images =  688


In [20]:

model1 = VGG16(weights="imagenet", include_top=False)
model1 = Sequential([
        layers.Conv2D(input_shape=(IMG_SIZE,IMG_SIZE,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
        layers.MaxPool2D(pool_size=(2,2)),
    
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.MaxPool2D(pool_size=(2,2)),
    
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Dropout(0.3),
        layers.MaxPool2D(pool_size=(2,2)),
    
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Dropout(0.2),
        layers.MaxPool2D(pool_size=(2,2)),
    
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
        layers.Dropout(0.1),
        layers.MaxPool2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(units=256,activation="relu"),
        layers.Dense(units=256,activation="relu"),
        layers.Dense(units=6, activation="softmax")
])
model1.compile(
    optimizer= tf.keras.optimizers.Adam(learning_rate = 0.00001),
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

model1.fit(X_train, y_train, validation_data = (X_val, Y_val), epochs = 30)
#model1.fit(X_train, y_train, epochs = 20)
#model1.save('model.h5')
    


Train on 3000 samples, validate on 180 samples
Epoch 1/30
3000/3000 [==============================] - 14s 5ms/sample - loss: 1.4974 - acc: 0.4070 - val_loss: 1.3371 - val_acc: 0.5556
Epoch 2/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 1.1292 - acc: 0.5590 - val_loss: 1.1269 - val_acc: 0.6389
Epoch 3/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 0.9636 - acc: 0.6423 - val_loss: 1.0437 - val_acc: 0.6722
Epoch 4/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 0.8351 - acc: 0.6940 - val_loss: 0.9920 - val_acc: 0.6778
Epoch 5/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 0.7296 - acc: 0.7350 - val_loss: 0.9743 - val_acc: 0.6556
Epoch 6/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 0.6632 - acc: 0.7680 - val_loss: 0.9115 - val_acc: 0.6889
Epoch 7/30
3000/3000 [==============================] - 13s 4ms/sample - loss: 0.5940 - acc: 0.7933 - val_loss: 0.8612 - val_acc: 0

In [ ]:
# os.remove("/kaggle/working/output.csv")
# os.remove("/kaggle/working/model.h5")
# os.remove("/kaggle/working/validation_data.npy")
# os.remove("/kaggle/working/train_data.npy")
# os.remove("/kaggle/working/test_data.npy")
# os.remove("/kaggle/working/best.csv")
# os.remove("/kaggle/working/vgg.csv")
# shutil.rmtree('/kaggle/working/train')
# shutil.rmtree('/kaggle/working/validation_data')
shutil.rmtree('/kaggle/working')

In [21]:
def test(): 
  Y_test=model1.predict(X_test)
  print(len(X_test))
  prediction=[]
  for i in Y_test:
    prediction.append(np.argmax(i))
  data={'image_name':column1,'label':prediction}
  print(len(column1),len(prediction))
  df=pd.DataFrame(data)
  df.to_csv('best.csv',index=False)
  print(df)

test()

688
688 688
    image_name  label
0      623.jpg      0
1      208.jpg      5
2      473.jpg      0
3      333.jpg      1
4      537.jpg      1
..         ...    ...
683    364.jpg      3
684     90.jpg      3
685    599.jpg      4
686     25.jpg      1
687    147.jpg      3

[688 rows x 2 columns]
